In [2]:
from google.colab import files
uploaded=files.upload()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
df = pd.read_csv('wine(1).csv')

Saving wine(1).csv to wine(1).csv


In [18]:
print("Размерность:",df.shape[0],"x",df.shape[1])
print("___________")
print("Типы данных по колонкам:")
print(df.dtypes)
print("___________")
print("Пропуски по колонкам")
print(df.isna().sum())

Размерность: 1599 x 12
___________
Типы данных по колонкам:
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                  object
dtype: object
___________
Пропуски по колонкам
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [ ]:
#Пропусков данных нет, следовательно балансировать классы не требуется
#Последний столбец принимает значения только "good" или "bad", следовательно классификация бинарная
#Категориальных признаков нет (не считая вектор ответа)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X = df.drop(['quality'], axis=1)
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
answers_pred = model.predict(X_test)
print(accuracy_score(y_test,answers_pred))
#Итоговая точность - 73%, результат терпимый, но в случае, если бы данных было больше, результат оказался бы лучше

0.73


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [20]:
[print(round(coef,3), end = " + ") for coef in model.coef_[0]]
print(round(model.intercept_[0],3))
#Наибольшее влияние оказал параметр "volatile acidity"

-0.039 + -2.744 + -0.715 + -0.019 + -1.064 + 0.029 + -0.02 + -1.04 + -1.684 + 1.8 + 0.924 + -1.407


In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()
Xtr_st = scaler.fit_transform(X_train)
Xte_st = scaler.fit_transform(X_test)
model = LogisticRegression()
model.fit(Xtr_st, y_train)
answers_pred = model.predict(Xte_st)
print('Стандартизация:',accuracy_score(y_test,answers_pred))
scaler = MinMaxScaler([0,1])
Xtr_no = scaler.fit_transform(X_train)
Xte_no = scaler.fit_transform(X_test)
model = LogisticRegression()
model.fit(Xtr_no, y_train)
answers_pred = model.predict(Xte_no)
print('Нормализация:',accuracy_score(y_test,answers_pred))
#С масштабированием данных результат вышел почти такой же, как и без него, что напрямую говорит нам о малом влиянии диапазона значений для логистической регрессии

Стандартизация: 0.7275
Нормализация: 0.7375


In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
scores = []
for i in range(1,15):
    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test) 
    scores.append(knn.score(X_test,y_test))
print(scores)

[0.6475, 0.5875, 0.62, 0.5925, 0.62, 0.5975, 0.5975, 0.5975, 0.605, 0.595, 0.6025, 0.5875, 0.6, 0.5975]


In [23]:
knn = KNeighborsClassifier(1)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test,y_pred)
#Результат вышел хуже - 65%

0.6475

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
scores = []
for i in range(1,15):
    knn = KNeighborsClassifier(i)
    knn.fit(Xtr_st,y_train)
    answers = knn.predict(Xte_st) 
    scores.append(knn.score(Xte_st,y_test))
print(scores)
scores = []
for i in range(1,15):
    knn = KNeighborsClassifier(i)
    knn.fit(Xtr_no,y_train)
    answers = knn.predict(Xte_no) 
    scores.append(knn.score(Xte_no,y_test))
print(scores)

[0.75, 0.7125, 0.725, 0.7075, 0.7075, 0.7175, 0.6925, 0.7225, 0.705, 0.7225, 0.725, 0.7075, 0.7175, 0.7225]
[0.6875, 0.7075, 0.7175, 0.7375, 0.7225, 0.7325, 0.7125, 0.7125, 0.7125, 0.7125, 0.7, 0.7025, 0.71, 0.72]


In [25]:
knn = KNeighborsClassifier(1)
knn.fit(Xtr_st,y_train)
y_pred = knn.predict(Xte_st)
print('Стандартизация:',accuracy_score(y_test,y_pred))
knn = KNeighborsClassifier(4)
knn.fit(Xtr_no,y_train)
y_pred = knn.predict(Xte_no)
print('Нормализация:',accuracy_score(y_test,y_pred))
#С нормализацией результат вышел заметно лучше. Делаем вывод, что метод ближайших соседей слабо эффективен, если не следить на масштабированием, но неплохо работает в обратном случае

Стандартизация: 0.75
Нормализация: 0.7375


**Итого у нас:**

Логистическая регрессия:

1.   Без масштабирования данных - 73%
2.   С масштабированием данных - 73% (станд.), 74% (норм.)

Метод ближайших соседей:
1.   Без масштабирования данных - 65%
2.   С масштабированием данных - 75% (станд.), 74% (норм.)

*Метод ближайших соседей с применением масштабирования данных (стандартизация) оказался наиболее эффективен*

